In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from boruta import BorutaPy
from sklearn.feature_selection import RFE, SelectKBest, SelectFromModel, f_regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import lightgbm as lightgbm

# import lightgbm
import numpy as np
# import xgboost
from sklearn.ensemble import  RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lasso, ElasticNet, BayesianRidge
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import VotingClassifier


In [ ]:
CURR_DIR = './public_data/'

# FUNCTIONS

In [78]:
def get_dummies_from_value_in_column(column_name, train_df, test_df):
    train_df[column_name].fillna(value='no_info', inplace=True)
    test_df[column_name].fillna(value='no_info', inplace=True)

    train_column = train_df[[column_name]]
    test_column = test_df[[column_name]]   
    all_data = pd.concat([train_column, test_column])

    for column in all_data.select_dtypes(include=[np.object]).columns:
        cat_type = CategoricalDtype(categories=all_data[column].unique(), ordered=True)
        train_column[column].astype(cat_type)
        test_column[column].astype(cat_type)
    
    onehot_train = pd.get_dummies(train_column, prefix=column_name)
    onehot_test = pd.get_dummies(test_column, prefix=column_name)
    
    train_df.drop(columns=[column_name], inplace=True)
    test_df.drop(columns=[column_name], inplace=True)
    
    return pd.merge(train_df, onehot_train, left_index=True, 
                    right_index=True), pd.merge(test_df, onehot_test, left_index=True, right_index=True)

def get_scaled_column(column_name, train_df, test_df): 
    scaler = MinMaxScaler()
    train_df_scaled = scaler.fit_transform(train_df[[column_name]].values.reshape(-1,1))
    test_df_scaled = scaler.transform(test_df[column_name].values.reshape(-1,1))
    train_df[column_name] = train_df_scaled
    test_df[column_name] = test_df_scaled
    return train_df, test_df

def box_categorical(columns, temp_dataset, Y_column):
    features = sorted(columns)

    ncols = 1
    nrows = math.ceil(len(features) / ncols)
    unused = (nrows * ncols) - len(features)

    figw, figh= ncols * 18, nrows * 12

    fig, ax = plt.subplots(nrows, ncols, figsize=(figw, figh))
    fig.subplots_adjust(hspace=0.2, wspace=0.2)
    ax = ax.flatten()
    for i in range(unused, 0, -1):
        fig.delaxes(ax[-i])

    for n, col in enumerate(features):
        ordering = temp_dataset.groupby(by=col)[Y_column].median().sort_values().index
        sns.boxplot(x=Y_column,y=col, data=temp_dataset, order=ordering, ax=ax[n], orient='h')
    plt.show()

def scatter_numerical(columns, temp_dataset, Y_column):
    features = sorted(columns)

    ncols = 1
    nrows = int(math.ceil(len(features) / ncols))
    unused = (nrows * ncols) - len(features)

    figw, figh = ncols * 20, nrows * 15

    fig, ax = plt.subplots(nrows, ncols, figsize=(figw, figh))
    fig.subplots_adjust(hspace = 0.5)
    ax = ax.flatten()

    for i in range(unused, 0, -1):
        fig.delaxes(ax[-i])

    for n, col in enumerate(features):
        if (n % 2 != 0):
            ax[n].yaxis.label.set_visible(False)
        ax[n].set_xlabel(col), ax[n].set_ylabel(Y_column)
        sns.scatterplot(x=col , y=Y_column, data=temp_dataset, hue=Y_column, s=150, legend=False, ax=ax[n])
    plt.show()    
    
def scale_and_select_features(features, y):

    def cor_selector(X, y):
        feature_names = X.columns.tolist()
        cor_list = []
        for i in X.columns.tolist():
            cor = np.corrcoef(X[i], y)[0, 1]
            cor_list.append(cor)
        cor_list = [0 if np.isnan(i) else i for i in cor_list]
        cor_feature = X.iloc[:, np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
        cor_support = [True if i in cor_feature else False for i in feature_names]
        return cor_support

    def _f_regr(X, y):
        X_norm = MinMaxScaler().fit_transform(X)
        chi_selector = SelectKBest(f_regression, k='all')
        chi_selector.fit(X_norm, y)
        chi_support = chi_selector.get_support()
        return chi_support

    def rfe(X, y):
        X_norm = MinMaxScaler().fit_transform(X)
        rfe_selector = RFE(estimator=RandomForestRegressor(n_estimators=10), n_features_to_select=100, step=10, verbose=5)
        rfe_selector.fit(X_norm, y)
        rfe_support = rfe_selector.get_support()
        return rfe_support

    def embedded_rf(X, y):
        embeded_rf_selector = SelectFromModel(RandomForestRegressor(n_estimators=10), threshold='1.25*median')
        embeded_rf_selector.fit(X, y)
        embeded_rf_support = embeded_rf_selector.get_support()
        return embeded_rf_support

    def embedded_lgbm(X, y):
        lgbc = lightgbm.LGBMRegressor(objective='regression', num_leaves=5,
                                        learning_rate=0.05, n_estimators=5,
                                        max_bin=55, bagging_fraction=0.8,
                                        bagging_freq=5, feature_fraction=0.2319,
                                        feature_fraction_seed=9, bagging_seed=9,
                                        min_data_in_leaf=6, min_sum_hessian_in_leaf=11)
        embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
        embeded_lgb_selector.fit(X, y)
        embeded_lgb_support = embeded_lgb_selector.get_support()
        return embeded_lgb_support

    def extra_trees(X, y):
        model = ExtraTreesRegressor(n_estimators=10)
        model.fit(X, y)
        support = np.array(model.feature_importances_ > 0.01)
        return support

    def boruta(X, y):
        X_norm = MinMaxScaler().fit_transform(X)
        rf = RandomForestRegressor()
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2)
        feat_selector.fit_transform(X_norm, y)
        return feat_selector.support_

    def select_top_mostly_selected_features():
        supports = [fn(features, y) for fn in [cor_selector, _f_regr, rfe, embedded_rf, embedded_lgbm, extra_trees, boruta]]

        feature_selection_df = pd.DataFrame(
            {'Feature': features.columns.tolist(), 'Pearson': supports[0],  'F_regr': supports[1],
             'RFE': supports[2], 'Random Forest': supports[3], 'LightGBM': supports[4], 'ExtraTrees': supports[5], 'Boruta': supports[6]})

        feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
        print(feature_selection_df)

        max_total = math.ceil(len(supports)*0.5) + 1
        feature_selection_df = feature_selection_df.sort_values(['Total', 'Feature'], ascending=False)
        feature_selection_df.index = range(1, len(feature_selection_df) + 1)
        feature_names = feature_selection_df[feature_selection_df['Total'] >= max_total]['Feature'].to_list()
        return feature_names

    if features.shape[0] > 0:
        feature_names = select_top_mostly_selected_features()
        features = features[feature_names]
        features = pd.DataFrame(features, columns=features.columns)
    return features, y

# def return_inner_models():

#     models = []

#     ### random forest model
#     rf_model = RandomForestRegressor(max_depth=5, random_state=42, n_estimators=5,
#                                      verbose=0)  # Train the model on training data
#     models.append(("rf", rf_model))
    
#     lr_model = LinearRegression()  
#     models.append(("lr", lr_model))

# #     model = GridSearchCV(
# #         estimator=SVR(kernel='rbf'),
# #         param_grid={
# #             'C': [0.1, 1, 100, 1000],
# #             'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
# #             'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
# #         },
# #         cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
# #     grid_results = model.fit(features, errors.ravel())
# #     best_params = grid_results.best_params_

#     ### svr model
#     svr_model = SVR(kernel='rbf',
#                     coef0=0.1, shrinking=True,
#                     tol=0.001, cache_size=200, verbose=False, max_iter=-1)
#     models.append(("svr", svr_model))
# # FUNKCJE
# #     ### xgboost
# #     xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
# #                                      learning_rate=0.05, max_depth=5,
# #                                      min_child_weight=1.7817, n_estimators=5,
# #                                      reg_alpha=0.4640, reg_lambda=0.8571,
# #                                      subsample=0.5213, silent=1,
# #                                      random_state=7, nthread=-1,
# #                                      tree_method='hist', predictor='cpu_predictor')
# #     models.append(("xgb", xgb_model))

#     ### lasso model
#     lasso_model = Lasso(alpha=0.0005, random_state=1, max_iter=10e5)
#     models.append(("lasso", lasso_model))

#     ### elastic net model
#     en_model = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)
#     models.append(("en", en_model))

#     ### krr model
#     krr_model = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
#     models.append(("krr", krr_model))

#     ### bayesian ridge
#     bayesian_ridge = BayesianRidge()
#     models.append(("br", bayesian_ridge))

#     ### lgbm model
#     lgbm_model = lightgbm.LGBMRegressor(objective='regression', num_leaves=5,
#                                         learning_rate=0.05, n_estimators=5,
#                                         max_bin=55, bagging_fraction=0.8,
#                                         bagging_freq=5, feature_fraction=0.2319,
#                                         feature_fraction_seed=9, bagging_seed=9, silent = True)
#     models.append(("lgbm",lgbm_model))
#     return models

def return_inner_models(features):

    models = []

    ### random forest model
    rf_model = RandomForestClassifier(max_depth=5, random_state=42, n_estimators=5,
                                     verbose=0)  # Train the model on training data
    models.append(("rf", rf_model))

    # model = GridSearchCV(
    #     estimator=SVC(kernel='rbf'),
    #     param_grid={
    #         'C': [0.1, 1, 100, 1000],
    #         'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
    #     },
    #     cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    # grid_results = model.fit(features, errors.ravel())
    # best_params = grid_results.best_params_
    #
    # ### svr model
    # svr_model = SVC(kernel='rbf', C=best_params["C"],
    #                 gamma=best_params["gamma"],
    #                 coef0=0.1, shrinking=True,
    #                 tol=0.001, cache_size=200, verbose=False, max_iter=-1)
    # models.append(("svr", svr_model))

    ### xgboost
    xgb_model = xgboost.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468,
                                     learning_rate=0.05, max_depth=5,
                                     min_child_weight=1.7817, n_estimators=5,
                                     reg_alpha=0.4640, reg_lambda=0.8571,
                                     subsample=0.5213, silent=1,
                                     random_state=7, nthread=-1,
                                     tree_method='hist', predictor='cpu_predictor')
    models.append(("xgb", xgb_model))

    ### naive bayes
    gnb = GaussianNB()
    models.append(('gnb', gnb))

    ### ridge
    ridge = RidgeClassifier()
    models.append(('ridge', ridge))

    ### logistic regression
    # logistic = LogisticRegression()
    # models.append(('logistic', logistic))

    ### mlp
    mlp = MLPClassifier(alpha=1, max_iter=1000)
    models.append(("mlp", mlp))

    ### knn
    knn = KNeighborsClassifier(n_neighbors=3)
    models.append(("knn", knn))

    ### adaboost
    ada = AdaBoostClassifier(n_estimators=100, random_state=0)
    models.append(("ada", ada))

    ### decision treee
    dtree = DecisionTreeClassifier(max_depth=5)
    models.append(("dtree", dtree))

    ### lgbm model
    lgbm_model = lightgbm.LGBMClassifier(num_leaves=5,
                                        learning_rate=0.05, n_estimators=5,
                                        max_bin=55, bagging_fraction=0.8,
                                        bagging_freq=5, feature_fraction=0.2319,
                                        feature_fraction_seed=9, bagging_seed=9,
                                        min_data_in_leaf=6, min_sum_hessian_in_leaf=11)
    models.append(("lgbm",lgbm_model))
    return models

# READ DATA

In [52]:
building_ownership = pd.read_csv(CURR_DIR + 'building_ownership.csv')
building_structure = pd.read_csv(CURR_DIR + 'building_structure.csv')
train = pd.read_csv(CURR_DIR + 'train.csv')
test = pd.read_csv(CURR_DIR + 'test.csv')
ward_demographic_data = pd.read_csv(CURR_DIR + 'ward_demographic_data.csv' )

In [53]:
merged_df = pd.merge(building_ownership, building_structure, on='building_id', how='left')
merged_df.drop(columns=['district_id_x', 'vdcmun_id_x', 'ward_id_x'],inplace=True)
merged_df.rename(columns={'district_id_y':'district_id', 'vdcmun_id_y':'vdcmun_id', 'ward_id_y':'ward_id'}, inplace=True)
merged_df = pd.merge(merged_df, ward_demographic_data, on='ward_id', how='left')
train_df = pd.merge(merged_df, train, on='building_id', how='right')
test_df = pd.merge(merged_df, test, on='building_id', how='right')

In [54]:
print(merged_df.shape)
print(building_ownership.shape)
print(building_structure.shape)
print(train_df.shape)
print(test_df.shape)
# print(test.shape)

(737054, 42)
(737054, 17)
(737054, 26)
(515937, 51)
(221117, 50)


# FILL IN MISSING DATA

Train

In [55]:
pd.set_option('display.max_columns', 200)
train_df[train_df.isnull().any(axis=1)]

,building_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,household_count,income_range_in_thousands,avg_hh_size,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,damage_grade
93652,93399,Private,NaN,0.0,0,0,0,0,0,0,0,0,0,0,20,2032,203201,2,3,725,18,Flat,Bamboo/Timber,Bamboo/Timber-Light roof,RC,Timber-Planck,Not attached,L-shape,0,0,0,0,0,1,0,0,0,0,0,124.0,0-10,5.0,1,0,0,0,1,0,0,1,1
280330,709812,Private,0.0,0.0,0,0,0,0,0,0,0,0,0,0,40,4035,403508,2,16,670,13,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Timber-Planck,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,4


Test

In [56]:
pd.set_option('display.max_columns', 200)
test_df[test_df.isnull().any(axis=1)]

,building_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,household_count,income_range_in_thousands,avg_hh_size,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall
46201,3331,Private,0.0,0.0,0,0,0,0,0,0,0,0,0,0,9,902,90203,2,26,260,14,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Other,TImber/Bamboo-Mud,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0,0,0,0,0,0,0
60495,12319,Private,0.0,0.0,0,0,0,0,0,0,0,0,0,0,10,1046,104605,2,40,300,12,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0,0,0,0,0,0,0


Filling in with median 

In [57]:
train_df = train_df.fillna(train_df.median())
test_df = test_df.fillna(train_df.median())

In [58]:
pd.set_option('display.max_columns', 200)
train_df[:2]

,building_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,household_count,income_range_in_thousands,avg_hh_size,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,damage_grade
0,44314,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,12,1236,123608,3,13,568,13,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,96.0,0-10,5.0,0,0,0,0,0,0,0,0,5
1,267748,Private,1.0,1.0,1,0,0,0,0,0,0,0,0,0,24,2414,241409,2,25,375,15,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,0,0,0,0,0,91.0,10-20,5.0,0,0,0,0,0,0,0,0,5


In [59]:
train_df.columns.shape

(51,)

# COLUMNS DECLARATION

In [60]:
target = 'damage_grade'
ids_columns = ['building_id', 'istrict_id', 'vdcmun_id', 'ward_id']
categorical_columns = ['legal_ownership_status', 'land_surface_condition', 'foundation_type','roof_type', 
                       'ground_floor_type', 'other_floor_type', 'position','plan_configuration', 'income_range_in_thousands' ]
onehot_columns = ['has_secondary_use', 'has_secondary_use_agriculture','has_secondary_use_hotel', 
                  'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school',
                  'has_secondary_use_industry', 'has_secondary_use_health_post','has_secondary_use_gov_office', 
                  'has_secondary_use_use_police','has_secondary_use_other', 'has_superstructure_adobe_mud',
                  'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
                  'has_superstructure_cement_mortar_stone','has_superstructure_mud_mortar_brick',
                  'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
                  'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
                  'has_superstructure_rc_engineered', 'has_superstructure_other','has_geotechnical_risk', 
                  'has_geotechnical_risk_fault_crack','has_geotechnical_risk_flood', 
                  'has_geotechnical_risk_land_settlement','has_geotechnical_risk_landslide', 
                  'has_geotechnical_risk_liquefaction','has_geotechnical_risk_other', 
                  'has_geotechnical_risk_rock_fall']
numerical_columns = ['count_families', 'count_floors_pre_eq', 'age_building', 'plinth_area_sq_ft',
                     'height_ft_pre_eq', 'household_count', 'avg_hh_size']

# PLOTS

In [ ]:
scatter_numerical(numerical_columns, train_df, target)

In [ ]:
def draw_histogram(query, x):
    figw, figh = 1 * 20, 1 * 15
    fig, ax = plt.subplots(1, 1, figsize=(figw, figh))
    sns.histplot(data=query, x=x, ax=ax, hue=target, palette='hls', multiple="stack")


age_building

In [ ]:
draw_histogram(train_df, 'age_building')

In [ ]:
print(train_df[train_df['age_building']>=999].shape) # około 0.5% ale nadal dość dziwne
print(train_df['age_building'].describe())
draw_histogram(train_df[train_df['age_building']<999], 'age_building')
### może warto rozważyć wyrzucenie? albo może to jest jakas miara, ze 'nie wiadomo'
train_df[train_df['age_building']>=999]
# i jeszcze kazdy ma inny damage

In [ ]:
train_df['age_building'].describe()

avg hh size

In [ ]:
draw_histogram(train_df, 'avg_hh_size')


In [ ]:
draw_histogram(train_df, 'count_families')
train_df['count_families'].describe()

In [ ]:
draw_histogram(train_df, 'count_floors_pre_eq')


In [ ]:
draw_histogram(train_df, 'height_ft_pre_eq')


In [ ]:
train_df[train_df['height_ft_pre_eq']>100]
# są jakieś 2 mega wysokie budynki i jeden jest niby na agriculture a drugi nie ma nic 
#kompletnie jako secondary use i ma duzy household count
# musiałyby mieć po 30 pięter

In [ ]:
draw_histogram(train_df, 'household_count')


In [ ]:
# draw_histogram(train_df, 'income_range_in_thousands')


In [ ]:
draw_histogram(train_df, 'plinth_area_sq_ft')


In [ ]:
train_df[train_df['plinth_area_sq_ft']>2000]['plinth_area_sq_ft'].unique()

inne kolumny

secondary use

In [ ]:
# train_df['has_secondary_use'] == q
q = train_df['has_secondary_use_agriculture'].astype(bool) | train_df['has_secondary_use_hotel'].astype(bool) | train_df['has_secondary_use_rental'].astype(bool) | train_df['has_secondary_use_institution'].astype(bool) | train_df['has_secondary_use_school'].astype(bool) | train_df['has_secondary_use_industry'].astype(bool) | train_df['has_secondary_use_health_post'].astype(bool) | train_df['has_secondary_use_gov_office'].astype(bool) | train_df['has_secondary_use_use_police'].astype(bool) | train_df['has_secondary_use_other']

In [ ]:
train_df[(train_df['has_secondary_use'].astype(bool) != q)]
# has secondary use

geotechnical risk

In [ ]:

# train_df['has_secondary_use'].astype(bool() == q
q = train_df['has_geotechnical_risk_fault_crack'].astype(bool) | train_df['has_geotechnical_risk_flood'].astype(bool) | train_df['has_geotechnical_risk_land_settlement'].astype(bool) | train_df['has_geotechnical_risk_landslide'].astype(bool) | train_df['has_geotechnical_risk_liquefaction'].astype(bool) | train_df['has_geotechnical_risk_other'].astype(bool) | train_df['has_geotechnical_risk_rock_fall'].astype(bool)
q2 = test_df['has_geotechnical_risk_fault_crack'].astype(bool) | test_df['has_geotechnical_risk_flood'].astype(bool) | test_df['has_geotechnical_risk_land_settlement'].astype(bool) | test_df['has_geotechnical_risk_landslide'].astype(bool) | test_df['has_geotechnical_risk_liquefaction'].astype(bool) | test_df['has_geotechnical_risk_other'].astype(bool) | test_df['has_geotechnical_risk_rock_fall'].astype(bool)

train_df[(train_df['has_geotechnical_risk'].astype(bool) != q)]
test_df[(test_df['has_geotechnical_risk'].astype(bool) != q2)]

# jedno gdzie się nie zgadza jest risk ale nie ma szczegółowego riska, po naprawie mozna wywalić? albo chociaz poanalizowac

In [61]:
train_df.at[445438,'has_geotechnical_risk'] = 0

# CORRELATION MAP

In [ ]:
corrmat = train_df.corr()
plt.subplots(figsize=(58, 58))
sns.heatmap(corrmat, vmax=1, square=True)

# BOX PLOTS

In [ ]:
box_categorical(categorical_columns, train_df, target)

# GROUP BY

In [ ]:
# train_df.groupby(['district_id']).count()

In [ ]:
test_df.groupby(['district_id']).count()['building_id']

In [ ]:
train_df.groupby(['vdcmun_id']).count()['building_id']

In [ ]:
train_df.groupby(['ward_id'])

# FEATURE ENGINEERING

In [62]:
train_df[:2]

,building_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,district_id,vdcmun_id,ward_id,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,household_count,income_range_in_thousands,avg_hh_size,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,damage_grade
0,44314,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,12,1236,123608,3,13,568,13,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,96.0,0-10,5.0,0,0,0,0,0,0,0,0,5
1,267748,Private,1.0,1.0,1,0,0,0,0,0,0,0,0,0,24,2414,241409,2,25,375,15,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,0,0,0,0,0,91.0,10-20,5.0,0,0,0,0,0,0,0,0,5


In [63]:
# dodajemy czy ma sąsiadów

In [64]:
train_df['neighbours'] = 'Yes'
train_df.loc[train_df['position']=='Not attached', 'neighbours'] = 'No'

test_df['neighbours'] = 'Yes'
test_df.loc[train_df['position']=='Not attached', 'neighbours'] = 'No'

new_categorical_cols = ['neighbours']
categorical_columns.append('neighbours')

In [65]:
# dodajemy czy to prosty plan czy nie

In [66]:
train_df['simple_plan_configuration'] = 'No'
train_df.loc[train_df['plan_configuration']=='Rectangular', 'simple_plan_configuration'] = 'Yes'
train_df.loc[train_df['plan_configuration']=='Square', 'simple_plan_configuration'] = 'Yes'

test_df['simple_plan_configuration'] = 'No'
test_df.loc[test_df['plan_configuration']=='Rectangular', 'simple_plan_configuration'] = 'Yes'
test_df.loc[test_df['plan_configuration']=='Square', 'simple_plan_configuration'] = 'Yes'

categorical_columns.append('simple_plan_configuration')
new_categorical_cols.append('simple_plan_configuration')

In [67]:
# czy ma więcej niż 2 piętra

In [68]:
train_df['more_than_two_floors'] = 'No'
test_df['more_than_two_floors'] = 'No'

train_df.loc[train_df['count_floors_pre_eq']>2, 'more_than_two_floors'] = 'Yes'
test_df.loc[test_df['count_floors_pre_eq']>2, 'more_than_two_floors'] = 'Yes'

categorical_columns.append('more_than_two_floors')
new_categorical_cols.append('more_than_two_floors')

In [69]:
# typ budynku

# train_df[train_df['legal_ownership_status']=='Private' & train_df['count_families']==1].value_counts()

# train_df['building_type'] = 'Other'
# train_df.query('legal_ownership_status == "Private" and count_families > 1' )['count_families'].value_counts()

In [70]:
# kategorie lat budynku
# pre 1950
# prenorm 1960-1965
# post_norm 1966-1995
# recent construction post 1995
train_df['year_of_construction'] = 2019 - train_df['age_building'] 
test_df['year_of_construction'] = 2019 - train_df['age_building'] 

train_df.loc[train_df['year_of_construction']>=1996, 'building_age_category'] = 'Recent construction'
train_df.loc[(train_df['year_of_construction']>=1950) & (train_df['year_of_construction']<=1965), 'building_age_category'] = 'Pre norm'
train_df.loc[(train_df['year_of_construction']>=1966) & (train_df['year_of_construction']<=1995), 'building_age_category'] = 'Post norm'
train_df.loc[train_df['year_of_construction']<1950, 'building_age_category'] = 'Pre 1950'

# train_df['building_age_category']

test_df.loc[test_df['year_of_construction']>=1996, 'building_age_category'] = 'Recent construction'
test_df.loc[(test_df['year_of_construction']>=1950) & (test_df['year_of_construction']<=1965), 'building_age_category'] = 'Pre norm'
test_df.loc[(test_df['year_of_construction']>=1966) & (test_df['year_of_construction']<=1995), 'building_age_category'] = 'Post norm'
test_df.loc[test_df['year_of_construction']<1950, 'building_age_category'] = 'Pre 1950'

categorical_columns.append('building_age_category')
new_categorical_cols.append('building_age_category')

In [71]:
# # ile cech ryzyk
train_df['number_of_geotechnical_risks'] = train_df[['has_geotechnical_risk_fault_crack','has_geotechnical_risk_flood', 
                  'has_geotechnical_risk_land_settlement','has_geotechnical_risk_landslide', 
                  'has_geotechnical_risk_liquefaction','has_geotechnical_risk_other', 
                  'has_geotechnical_risk_rock_fall']].sum(axis=1)
# train_df.drop(columns=['number_of)technological_risks'], inplace=True)
# train_df['number_']
train_df['number_of_geotechnical_risks_higher_than_0'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>0, 'number_of_geotechnical_risks_higher_than_0'] = 1
train_df['number_of_geotechnical_risks_higher_than_1'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>1, 'number_of_geotechnical_risks_higher_than_1'] = 1
train_df['number_of_geotechnical_risks_higher_than_2'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>2, 'number_of_geotechnical_risks_higher_than_2'] = 1
train_df['number_of_geotechnical_risks_higher_than_3'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>3, 'number_of_geotechnical_risks_higher_than_3'] = 1
train_df['number_of_geotechnical_risks_higher_than_4'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>4, 'number_of_geotechnical_risks_higher_than_4'] = 1
train_df['number_of_geotechnical_risks_higher_than_5'] = 0
train_df.loc[train_df['number_of_geotechnical_risks']>5, 'number_of_geotechnical_risks_higher_than_5'] = 1







# train_df[train_df['number_of_technological_risks']>0]
train_df.columns

Index(['building_id', 'legal_ownership_status', 'count_families',
       'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'district_id', 'vdcmun_id', 'ward_id',
       'count_floors_pre_eq', 'age_building', 'plinth_area_sq_ft',
       'height_ft_pre_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
    

In [72]:
# print(train_df.groupby(target).count())
balance_length = train_df.groupby(target).count().min()[0]
# balance_length
# grade_1
b1=train_df[train_df[target] == 1].sample(n=balance_length)
b2=train_df[train_df[target] == 2].sample(n=balance_length)
b3=train_df[train_df[target] == 3].sample(n=balance_length)
b4=train_df[train_df[target] == 4].sample(n=balance_length)
b5=train_df[train_df[target] == 5].sample(n=balance_length)
balanced_train_df = pd.concat([b1,b2,b3,b4,b5]).copy()
balanced_test_df = test_df.copy()
# balance_length*5

In [ ]:
box_categorical(['number_of_geotechnical_risks',
       'number_of_geotechnical_risks_higher_than_0',
       'number_of_geotechnical_risks_higher_than_1',
       'number_of_geotechnical_risks_higher_than_2',
       'number_of_geotechnical_risks_higher_than_3',
       'number_of_geotechnical_risks_higher_than_4',
       'number_of_geotechnical_risks_higher_than_5'], balanced_train_df, target)

In [ ]:
box_categorical(categorical_columns, balanced_train_df, target)

In [ ]:
box_categorical(new_categorical_cols, train_df, target)

In [ ]:
# group ward
# draw_histogram(balanced_train_df, 'number_of_geotechnical_risks_higher_than_1')

In [ ]:
# train_df['year_of_construction']
draw_histogram(train_df, 'year_of_construction')

In [ ]:
train_df.query('legal_ownership_status == "Private" and count_families == 0' )#.value_counts()

# FEATURE SCALING

In [73]:
for n in numerical_columns:
    train_df, test_df = get_scaled_column(n, train_df, test_df)
    
for c in categorical_columns:
    train_df, test_df = get_dummies_from_value_in_column(c, train_df, test_df)
        
for n in numerical_columns:
    balanced_train_df, balanced_test_df = get_scaled_column(n, balanced_train_df, balanced_test_df)
    
for c in categorical_columns:
    balanced_train_df, balanced_test_df = get_dummies_from_value_in_column(c, balanced_train_df, balanced_test_df)

In [75]:
train_df.shape

(515937, 104)

# PREPARE FEATURES

In [ ]:
import re
import math
pd.set_option('display.max_rows', 200)
selected_dataset = balanced_train_df#### nonans_dataset#noinfo_dataset #### tutaj usuwamy te rzedzy co maja duzo kolumn, ktore sa puste
# selected_dataset
# selected_dataset = selected_dataset.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
selected_dataset.reset_index(drop=True, inplace=True)
X, y = selected_dataset.loc[:, selected_dataset.columns != target], selected_dataset[[target]]
# y[y.isnull().any(axis=1)]
a, b = scale_and_select_features(X, y.iloc[:,0])
# selected_dataset
# y
X, y = a,b

Fitting estimator with 102 features.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


# TRAIN

In [ ]:
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import cross_val_score
from numpy import mean
import random
import sys
from numpy import std
# from sklearn.model_selection import cross_val_score

# X = np.array([[0, 0, 0], [1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]])
# y = np.array([0, 1, 2, 3, 4])
# X, y = selected_dataset.loc[:, selected_dataset.columns != Y_column], selected_dataset[Y_column]
# X = 
# X,y = f[0], f[1]
def RMSEcalc(a, b):
    return np.sqrt(metrics.mean_squared_error(a, b))
from sklearn.model_selection import KFold # import KFold
# # # X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) # create an array
# # # y = np.array([1, 2, 3, 4]) # Create another array
# # kf = KFold(n_splits=10) # Define the split - into 2 folds 
# # kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

# # X_train, X_test, y_train, y_test = train_test_split(X, y, 
# #                                                     test_size=0.1, random_state=0)

# # random
for i in range(1):
    kf = KFold(n_splits=10, random_state=random.randint(0, 2**32-1), shuffle=True)
#     kf = KFold(n_splits=20, random_state=5, shuffle=True)


    # # create model
    # # model = LinearRegression()
    model = VotingClassifier(return_inner_models())
    # # evaluate model
    # scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
    # # report performance
    # scores = -scores
    # print('RMSE: %.3f (%.3f)' % (mean(np.sqrt(scores)), std(np.sqrt(scores))))

    # from sklearn.model_selection import KFold
    # from sklearn.linear_model import LinearRegression

    # ## Define variables for the for loop

    # kf = KFold(n_splits=10)
    RMSE_sum=0
    RMSE_length=10
    # X = np.array(comm_df)
    # y = np.array(comm_target)
    r = []

    for loop_number, (train, test) in enumerate(kf.split(X)):
    #     print(train)
    #     print(train[0])
    #     print(y.loc[y.index.intersection(train)])
    #     print(y.loc[train[0]])
    #     print(y.loc[test,:])

        ## Get Training Matrix and Vector

        training_X_array = X.reindex(index=train)
        training_y_array = y.reindex(index=train)#.reshape(-1, 1)
    #     print(training_y_array)
        ## Get Testing Matrix Values

        X_test_array = X.reindex(index=test)#[test]
        y_actual_values = y.reindex(index=test)#[test]

        ## Fit the Linear Regression Model

    #     lr_model = LinearRegression().fit(training_X_array, training_y_array)
        model.fit(training_X_array, training_y_array)
        ## Compute the predictions for the test data

        prediction = model.predict(X_test_array)      
        crime_probabilites = np.array(prediction)   

        ## Calculate the RMSE

        RMSE_cross_fold = RMSEcalc(crime_probabilites, y_actual_values)
        r.append(RMSE_cross_fold)
        ## Add each RMSE_cross_fold value to the sum

        RMSE_sum=RMSE_cross_fold+RMSE_sum

    ## Calculate the average and print    

    RMSE_cross_fold_avg=RMSE_sum/RMSE_length

    print('The Mean RMSE across all folds is',RMSE_cross_fold_avg)
    print(mean(r))
#     print(r)